In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  8 15:45:44 2022

@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
#import pyautogui
from selenium.webdriver.common.action_chains import ActionChains
#import shutil
#import glob
import os
import warnings
#import itertools
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import sys
warnings.filterwarnings("ignore")


In [2]:
#open a new window, load the website and logs in

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')


RL = 'https://www.publicpower.org/event/group/conferences-meetings'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [3]:
driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/div[1]/nav[2]/ul/li[4]/a').click()

In [4]:
litems=driver.current_url+'/list'
driver.get(litems)

In [5]:
html0 = driver.page_source
soup0 = bs(html0,'lxml')
pager= soup0.find('li',{'class':'pager__item pager__item--last'}).find('a')['href'][-1]
pager

'1'

In [8]:
APPA = []
for i in range(0,int(pager)+1):
    url=litems+f'?page={i}'
    driver.get(url)
    html = driver.page_source
    soup = bs(html,'lxml')
    allItems = soup.find_all('div',{'class':'node node--type-event node--view-mode-list-view ds-1col clearfix'})
    for ja in allItems:
        #try:
        tit=ja.find('div',{'class':'field-wrapper field field-node--node-title field-name-node-title field-type-ds field-label-hidden'}).text.replace('\n','')
        #except:
         #   tit=''
          #  pass
        web=ja.find('a')['href']
        link='https://www.publicpower.org'+web
        driver.get(link)
        event_web=driver.current_url
        html1 = driver.page_source
        soup1 = bs(html1,'lxml')
        when_where = soup1.find('div',{'class':'wrapper-lede'})
        funders= soup1.find('section',{'class':'block-event-sponsors small-12 column clearfix'})
        footer=soup1.find('footer',{'class':'row main-footer'})
        information=soup1.find('section',{'class':'block-appa-foundation-content small-12 column clearfix'})
        ###############################################################
        title=when_where.find('div',{'class':'field-wrapper field field-node--node-title field-name-node-title field-type-ds field-label-hidden'}).text.replace('\n','').strip()
        
        ################################################
        try:
            spon=funders.find_all('div',{'class':'owl-item show'})
        except:
            try:
                spon=footer.find_all('div',{'class':'owl-item show'})
            except:
                spon=[]
        sponsors=[]
        for za in spon:
            try:
                pi=za.find('img')['alt']
            except:
                pi=''
            sponsors.append(pi)
        if len(sponsors)>=1:
            spons=sponsors
            sponsor=json.dumps(spons)
        else:
            sponsor=''
        #########################################
        try:
            event_info=when_where.find('div',{'class':'field-wrapper body field field-node--body field-name-body field-type-text-with-summary field-label-hidden'}).text.replace('\n','').strip()
        except:
            event_info=''
        ##################################################
        try:
            ven=when_where.find('div',{'class':'field-wrapper field field-node--field-loc-name field-name-field-loc-name field-type-string field-label-hidden'}).text.replace('\n','').strip()
            on_off='0'

        except:
            ven=''    
            on_off='1'
        #########################################
        try:
            dateill=when_where.find('div',{'class':'field-wrapper field field-node--field-event-date-range field-name-field-event-date-range field-type-daterange field-label-hidden'}).text.replace('\n','').strip()
            date_=dateill.split(',')
            year=date_[-1].strip()
            mon_day=date_[0]
            if '-' in mon_day:
                start_=mon_day.split('-')[0].strip()+', '+year
                #get the month as digit by converting to datetime
                start_dt_obj = datetime.strptime(start_, '%B %d, %Y')
                start_date=start_dt_obj.strftime('%Y-%m-%d')
                #start_date=start_dt.date()

                #get month
                obj_mn=start_dt_obj.month
                en_=mon_day.split('-')[-1].strip()

                if len(en_)>2:
                    end_=en_+', '+year
                    end_dt_obj = datetime.strptime(end_, '%B %d, %Y')
                    end_date=end_dt_obj.strftime('%Y-%m-%d')
                    #end_date=end_dt.date()
                else:
                    end_=str(obj_mn)+' '+en_+', '+year
                    end_dt_obj = datetime.strptime(end_, '%m %d, %Y')
                    end_date=end_dt_obj.strftime('%Y-%m-%d')
                    #end_date=end_dt.date()
            else:
                start_=end_=dateill
                start_dt_obj = end_dt_obj=datetime.strptime(start_, '%B %d, %Y')
                start_date=end_date=start_dt_obj.strftime('%Y-%m-%d')
                #start_date=start_dt.date()
        except:
            start_date=end_date=''
        ############################################################
        try:
            headin=information.find('ul',{'class':'tabs event-node-tabs'}).text.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n').split('\n')[1:-1]
            if 'Event Info' in headin:
                #drum=index('Hotel')
                index = headin.index("Event Info")
                dan=driver.current_url
                if '#event-info' in dan:
                    #cur=dan.split('#')[0]
                    drink=dan#cur+f'#event-{index}'
                elif '#' in dan:
                    cur=dan.split('#')[0]
                    drink=cur+'#event-info'
                driver.get(drink)

                #timing
                ken='/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[1]/div/div/div/h3[1]'
                #datetime
                eze='/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[1]/div/div/div/p[1]'
                #condition
                try:
                    if driver.find_element_by_xpath(ken).text=='Timing':
                        date22t=driver.find_element_by_xpath(eze).text


                    else:
                        #print('dunk')
                        date22t=''
                except:
                    #print('tri')
                    date22t=''
            else:
                #print('bi')
                date22t=''
        except:
            date22t=''
        ###
        if '\n' in date22t:
            whole=date22t.split('\n')
            who=whole[1].replace('.','')
            if 'each day' in who:
                dura=who.split('each day')[0]
                sup=dura.split('–')
                tim_spl=[(q.strip().split(' ')[0])+(q.strip().split(' ')[1].upper()) for q in sup]

            elif ': ' in who:
                dura=who.split(': ')[1]
                sup=dura.split('–')
                #tim_spl=[q.strip() for q in sup]
                tim_spl=[(q.strip().split(' ')[0])+(q.strip().split(' ')[1].upper()) for q in sup]

            elif ' :' in who:
                dura=who.split(' :')[0]
                sup=dura.split('–')
                #tim_spl=[q.strip() for q in sup]
                tim_spl=[(q.strip().split(' ')[0])+(q.strip().split(' ')[1].upper()) for q in sup]

            else:
                sup=who.split('–')
                tim_spl=[(q.strip().split(' ')[0])+(q.strip().split(' ')[1].upper()) for q in sup]
            time_st={'type': 'general',
                      'start_time':tim_spl[0],
                      'end_time': tim_spl[1],
                      'timezone': '',
                      'days': 'all'}
            time_=json.dumps(time_st)
        else:

            time_=''
        #########################################################
        fizzy=information.find('ul',{'class':'tabs event-node-tabs'}).text.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n').split('\n')[1:-1]
        if 'Hotel' in fizzy:
            #drum=index('Hotel')
            stef = fizzy.index("Hotel")
            dan=driver.current_url
            if '#' in dan:
                cur=dan.split('#')[0]
                drink=cur+f'#event-{stef}'
            else:
                drink=dan+f'#event-{stef}'
            driver.get(drink)
            try:
                if driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[3]/div/div/div/h3[1]').text=='Hotel':
                    Hotel=driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[3]/div/div/div/p[2]').text.replace(' \n',', ').replace('\n',', ').strip()
        #         elif driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[6]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/h3[1]').text=='Hotel':
        #         Hotel=driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[6]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/p[2]').text.replace(' \n',' ').replace('\n',', ').strip()
            except:
                try:
        #             
                    if driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[6]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/h3[1]').text=='Hotel':
                        Hotel=driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[6]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/p[2]').text.replace(' \n',', ').replace('\n',', ').strip()
                except:
                    try:
                        if driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/h2').text=='Hotel':
                            Hotel=driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[5]/div/main/section/div/section[1]/div[2]/div/div[4]/div/div/div/p[1]').text.replace(' \n',', ').replace('\n',', ').strip()
                    except:
                        Hotel=''
        else:
            #print('honey')
            Hotel=''
        if len(Hotel)>1:
            venue=Hotel
        else:
            if len(ven)>1:
                venue=ven
            else:
                venue=''

        #########################################################
        
        
        
        
        #########################################################
        city=ven
        if city != '':
            country='USA'
        else:
            country=''
        ########################################################
        agendalist=''
        ticket_list=''
        Speakerlist=''
        logo=''
        type_=''
        category=''
        mail_=json.dumps(['EducationInfo@PublicPower.org'])
        org_name='American Public Power Association'
        org_web='https://www.publicpower.org/'
        try:
            org_pro=footer.find('section',{'class':'block-abouttheamericanpublicpowerassociation-3 small-12 column clearfix'}).text.replace('\n','').strip()
        except:
            org_pro=''
        #########################################################
        def get_google_map_url(location):
            try:
                google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                google_map_url=driver.find_element_by_id('lu_map').click()
                time.sleep(1)
                google_map_url=driver.current_url
                return(google_map_url)
            except Exception as e:
                print(e)
                return("")
        
        if venue!='':
            googlePlaceUrl=get_google_map_url(venue)
        else:
            googlePlaceUrl=''
        
        #########################################################

        APPA.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                         org_pro,org_name,org_web,logo,sponsor,agendalist,
                         type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                         Speakerlist,on_off])
        
    

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B99943+2595139]
	Ordinal0 [0x00B2C9F1+2148849]
	Ordinal0 [0x00A24528+1066280]
	Ordinal0 [0x00A50FD4+1249236]
	Ordinal0 [0x00A511CB+1249739]
	Ordinal0 [0x00A7D812+1431570]
	Ordinal0 [0x00A6BA34+1358388]
	Ordinal0 [0x00A7BAF2+1424114]
	Ordinal0 [0x00A6B806+1357830]
	Ordinal0 [0x00A46086+1204358]
	Ordinal0 [0x00A46F96+1208214]
	GetHandleVerifier [0x00D3B232+1658114]
	GetHandleVerifier [0x00DF312C+2411516]
	GetHandleVerifier [0x00C2F261+560433]
	GetHandleVerifier [0x00C2E366+556598]
	Ordinal0 [0x00B3286B+2173035]
	Ordinal0 [0x00B375F8+2192888]
	Ordinal0 [0x00B376E5+2193125]
	Ordinal0 [0x00B411FC+2232828]
	BaseThreadInitThunk [0x76C2FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77787A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77787A6E+238]



In [10]:
APPA_dict=APPA


APPA_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=APPA_dict)
APPA_df.to_csv("APPA.tsv", sep = '\t',index=False)
